analysis

In [39]:
from uavsar_pytools.snow_depth_inversion import depth_from_phase, phase_from_depth

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import rioxarray as rxa
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl
import pandas as pd
import os

def get_site_dSWE(site_name, d1, d2, df):
    site = df[df.site_name == site_name]
    if not d1.tz:
        d1 = d1.tz_localize('UTC')
    if not d2.tz:
        d2 = d2.tz_localize('UTC')
    site_pair = site.loc[d1:d2, 'SWE']
    if len(site_pair) < 2:
        return np.nan
    dSWE = site_pair.iloc[-1] - site_pair.iloc[0]
    return dSWE

def add_delta_t(ds):
    t_deltas = [pd.Timedelta(t.values) for t in ds.time.diff(dim = 'time')]
    t_deltas.append(np.nan)
    ds = ds.assign_coords({'t_delta': ('time', t_deltas)})

    return ds

def add_dSWEs_dTime(ds, df):

    ds = add_delta_t(ds)

    dSWEs = []
    for sub_t in ds.time:
        start = pd.to_datetime(ds['time'].sel(time = sub_t).values)
        end = pd.to_datetime(start + ds['t_delta'].sel(time = sub_t).values)
        dSWE = get_site_dSWE(ds.attrs['site'].lower(), start, end, df)
        dSWEs.append(dSWE)
    
    ds = ds.assign_coords({'dSWE': ('time', dSWEs)})

    return ds

In [38]:
df = pd.read_parquet('/bsuhome/zacharykeskinen/uavsar-validation/data/insitu/snotel.parq')
df = df.reset_index(level= 1)
df = df.sort_index()
# loop through each flight pair and calculate rmse, r2, plot
lidar = None

lidar_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar')

for lidar_fp in lidar_dir.glob('*.all.nc'):

    print(lidar_fp)
    lidar = xr.open_dataset(lidar_fp)

    if isinstance(lidar.attrs['lidar_times'] , str):
        lidar.attrs['lidar_times'] = [lidar.attrs['lidar_times']]

    for t in lidar.attrs['lidar_times']:
        print(t)

        t = pd.to_datetime(t)
        ds = lidar.sel(time = slice(t - pd.Timedelta('180 days'), t))
        
        lidar_img = ds['lidar-sd'].sel(time = t)

        if len(ds.time) < 2:
            continue

        ds = ds.isel(time = slice(0, len(ds.time)-1))
        ds = add_dSWEs_dTime(ds, df)
        ds = ds.sel(time = ds.dSWE > 0) # more than 0.1 inches of swe
        # ds = ds.sel(time = ds.t_delta < pd.Timedelta('10 days'))

        if len(ds.time) < 2:
            continue

        os.makedirs(f"/bsuhome/zacharykeskinen/uavsar-validation/figures/lidar/summed/{ds.attrs['site']}", exist_ok = True)

        for img_type in ['int', 'unw']:

            out_fp = f"/bsuhome/zacharykeskinen/uavsar-validation/figures/lidar/summed/{ds.attrs['site']}/{img_type}-{ds.attrs['site']}-{(t).strftime('%Y-%m-%d')}.png"
            
            # if Path(out_fp).exists():
            #     continue

            fig, axes = plt.subplots(1, 3, figsize = (20, 10))

            sub232 = ds[f'dSD_232_{img_type}'].sel(band = 'VV').rolling(x = 5).mean().rolling(y = 5).mean()
            
            sub = sub232.sum(dim = 'time')

            inc = ds['232_inc'].isel(time = 0)

            lidar_img.plot(vmin = 0, vmax = 3, ax = axes[0])

            vmin, vmax = sub.quantile([0.2, 0.85]).values
            sub = sub
            sub.where(~lidar_img.isnull()).plot(vmin = vmin, vmax = vmax, ax = axes[1], cmap = 'viridis')
            # cor.where(~lidar_img.isnull()).plot(vmin = 0, vmax = 1, ax = axes[2])

            ax = axes[2]

            xs = lidar_img.values.ravel()
            ys = sub.values.ravel()

            xs_tmp = xs[(~np.isnan(xs)) & (~np.isnan(ys)) & (ys != 0)]
            ys = ys[(~np.isnan(xs)) & (~np.isnan(ys)) & (ys != 0)]
            xs = xs_tmp

            if len(ys) == 0:
                plt.clf()
                continue

            range = [[-.2, 5.5], [-.2, np.max(ys) + 0.3]]
            ax.hist2d(xs_tmp, ys, bins = 100, norm=mpl.colors.LogNorm(), cmap=mpl.cm.inferno, range = range)

            from scipy.stats import pearsonr
            r, p = pearsonr(xs, ys)
            ax.text(.01, .99, f'r: {r:.2}\nn = {len(ys):.2e}', ha = 'left', va = 'top', transform = ax.transAxes)

            plt.savefig(out_fp)
            plt.clf()

/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Banner.all.nc
2020-02-18T00:00:00.000000000
2021-03-15T00:00:00.000000000
/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Dry_Creek.all.nc
2020-02-19T00:00:00.000000000
/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Mores.all.nc
2020-02-09T00:00:00.000000000
2021-03-15T00:00:00.000000000


<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>